In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from adadamp import DaskBaseDamper

In [2]:
# model from https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
from functools import lru_cache
import numpy as np
import time

In [4]:
# this is bad practice
@lru_cache
def get_list_from_dataset(dataset):
    X = [d[0].reshape(-1, *d[0].size())[0][0][0] for d in dataset] # what does reshapse do and can i do this in the transform array?
    y = [d[1] for d in dataset] 
    return X, y

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
X, y = get_list_from_dataset(dataset) # ~ 23 seconds

In [6]:
# better
@lru_cache
def get_list_from_dataloader(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=1000)
    X = None
    y = None
    for (idx, batch) in enumerate(loader):
        if X == None:
            X = batch[0]
            y = batch[1]
        else:
            X = torch.cat((X, batch[0]),0)
            y = torch.cat((y, batch[1]))
    # need to use this to transform y before returning
    # -- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    return X, y

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
X, y = get_list_from_dataloader(dataset)

In [7]:
X.size() # is this supposed to be [60000, 28x28 = 728]?

torch.Size([60000, 1, 28, 28])

In [8]:
y.size()

torch.Size([60000])

In [9]:
def generate_stats(epochs=14):
    # params
    device = torch.device("cpu")
    log_interval = 10
    train_kwargs = {'batch_size': 64}
    # transform data
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    # load data
    dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
    #train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    
    data_load_start = time.time()
    X, y = get_list_from_dataloader(dataset1)
    y2 = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()
    print("Loaded data in", time.time() - data_load_start, "seconds")
    print("\tX.size() ==", X.size())
    print("\ty.size() ==", y.size())
    
    net = DaskBaseDamper(module=Net, loss=nn.NLLLoss, optimizer=optim.Adadelta, optimizer__lr=1.0, batch_size=64)
    #print(net.get_params())
    
    # train model
    stats_for_epochs = []
    examples = enumerate(dataset1)
    for epoch in range(1, epochs + 1):
        # ERROR when passing the dataset -> 1D target tensor expected, multi-target not supported
        # ----- occurs if i directly pass a dataset, or if i separate X and y first too
        _ = net.fit(X, y2) # Compre to MLP regressor from SKLearn
        loss = net.score(X, y2)
        
        stats = { "loss": score, "time": 0 }
        stats_for_epochs.append(stats)
        break
    return stats_for_epochs

In [10]:
stats = generate_stats(2)

Loaded data in 13.856945037841797 seconds
	X.size() == torch.Size([60000, 1, 28, 28])
	y.size() == torch.Size([60000])


ValueError: only one element tensors can be converted to Python scalars